In [2]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
import time
from selenium import webdriver
from datetime import datetime

In [3]:
!which chromedriver

/usr/local/bin/chromedriver


In [4]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
img_url ='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
twitter_url = 'https://twitter.com/marswxreport?lang=en'
facts_url = 'https://space-facts.com/mars/'
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [5]:
response = requests.get(url)
soup = bs(response.text, 'lxml')

In [6]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [7]:
browser.visit(img_url)
time.sleep(1)

In [8]:
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(1)

In [9]:
more_info = browser.find_link_by_partial_text('more info')
more_info.click()

In [10]:
image_page = browser.html
soup = bs(image_page, 'lxml')

In [11]:
image_link = soup.find('figure', class_='lede').find('img')['src']
image_link

'/spaceimages/images/largesize/PIA14254_hires.jpg'

In [12]:
featured_image_url = ("https://www.jpl.nasa.gov/" + image_link)
featured_image_url

'https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA14254_hires.jpg'

In [13]:
browser.visit(twitter_url)
twitter_page = browser.html
twitter_soup = bs(twitter_page, 'lxml')
time.sleep(1)

In [14]:
mars_weather = twitter_soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text
mars_weather

'Sol 2056 (May 19, 2018), Sunny, high 5C/41F, low -74C/-101F, pressure at 7.40 hPa, daylight 05:20-17:20'

In [15]:
read_mars = pd.read_html('https://space-facts.com/mars/')
df = pd.DataFrame.to_html(read_mars[0])
df

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    

In [16]:
browser.visit(usgs_url)
items = browser.find_by_css('a.product-item h3') 
time.sleep(1)

In [17]:
# div = browser.find_by_css('div.item')
# item = div.find_by_css('a.product-item h3').first
# time.sleep(1)
# item.click()

In [18]:
hemisphere = []

In [19]:
for i in range(len(items)):
    data = {}
    browser.find_by_css('a.product-item h3')[i].click()
    time.sleep(2)
    img_url = browser.find_link_by_text('Sample').first
    data['url'] = img_url['href']
    data['title'] = browser.find_by_css('h2.title').text
    hemisphere.append(data)
    browser.back()

In [20]:
hemisphere

[{'title': 'Cerberus Hemisphere Enhanced',
  'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [21]:
mars = pd.DataFrame(hemisphere)
mars

,title,url
0,Cerberus Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
1,Schiaparelli Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
2,Syrtis Major Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
3,Valles Marineris Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...


In [22]:
title = []
url = []
for i in range(len(hemisphere)):
        name = hemisphere[i]['title']
        link = hemisphere[i]['url']
        title.append(name)
        url.append(link)

In [23]:
title

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [24]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


In [25]:
db = client.mars_data_db
collection = db.mars_data

In [26]:
mars_data = {}

In [27]:
print('===================')
print(featured_image_url)

mars_data['featured_image']= featured_image_url

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA14254_hires.jpg


In [28]:
 print('===================')
print(mars_weather)
mars_data['weather'] = mars_weather

Sol 2056 (May 19, 2018), Sunny, high 5C/41F, low -74C/-101F, pressure at 7.40 hPa, daylight 05:20-17:20


In [46]:
print('===================')
# print(df)
df = df.replace('\n', '')
mars_data['facts'] = df
df

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>0</th>      <th>1</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    <

In [48]:
mars_data['hemisphere'] = hemisphere
    
print('===================')
print(hemisphere)
print('===================')
print(mars_data)

[{'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'title': 'Cerberus Hemisphere Enhanced'}, {'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'title': 'Schiaparelli Hemisphere Enhanced'}, {'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg', 'title': 'Syrtis Major Hemisphere Enhanced'}, {'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg', 'title': 'Valles Marineris Hemisphere Enhanced'}]
{'featured_image': 'https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA14254_hires.jpg', 'weather': 'Sol 2056 (May 19, 2018), Sunny, high 5C/41F, low -74C/-101F, pressure at 7.40 hPa, daylight 05:20-17:20', 'facts': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>0</th>      <th>1</th>    </tr>  </thead>  <tbody>    <tr>

In [49]:
collection.insert_one(mars_data)
